# Lab Assigment 2 Estimation of Apparent Motion

<p style="text-align:left;">
    José Pedro Cruz
    <span style="float:right;">
        up201504646
    </span>
</p>
<p style="text-align:left;">
    Martinho Figueiredo
    <span style="float:right;">
        up201506179
    </span>
</p>
<p style="text-align:left;">
    Nuno Nascimento
    <span style="float:right;">
        up201907933
    </span>
</p>


[![GitHub](https://img.shields.io/badge/github-%23121011.svg?style=for-the-badge&logo=github&logoColor=white)](https://github.com/martinhofigueiredo/VC)



## Scheme
```mermaid
flowchart LR
    A[Ingest Footage] -->|mp4 or mpegs| B{Multi Channel?}
    B -->|Yes| C[Split into Channels]
    B -->|No| D[Gray]-->H
    C --> E[R]-->H
    C --> F[G]-->H
    C --> G[B]-->H
    H{Multi Resolution?}-->|Yes|I[n Pyramid DownSampling Average]-->K
    H-->|No|J[One Shot]-->K
    K{algo} -->|HornShunck| M((.flo file))
    K -->|LucasKanade| M
```


## Benchmarking

```mermaid
flowchart LR
    GT[Ground truth] --> B{BenchMark}
    A(Flow Calculated) --> B
    B --> AE(Angular Error and Std Dev)
    B --> EE(Endpoint Error and Std Dev)
```


## Visualization
```mermaid
flowchart LR
    A[import .flo file]-->C{Type}
    C-->VF[Vector Field]
    C-->MF[Middlebury Flow]
    
```

# Setup

In [2]:
%pip install scipy
%pip install flowvid
%pip install imageio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import numpy as np
import yaml
import os
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from scipy.ndimage.filters import convolve as filter2
import struct
import argparse
import math
%matplotlib widget


/tmp/ipykernel_22256/130197631.py:7: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import convolve as filter2


## Import Dataset

In [4]:
# IMport dataset with public ground truth
%pip install wget
import os
import subprocess
import wget
import zipfile

# Specify the URLs of the files to download
url1 = "https://vision.middlebury.edu/flow/data/comp/zip/other-gt-flow.zip"
url2 = "https://vision.middlebury.edu/flow/data/comp/zip/other-color-allframes.zip"
url3 = "https://vision.middlebury.edu/flow/data/comp/zip/eval-color-allframes.zip"
# Specify the destination folder to store the downloaded files
destination_folder = "dataset/"

# Download the files using wget
file1 = wget.download(url1, out=destination_folder)
file2 = wget.download(url2, out=destination_folder)
file3 = wget.download(url3, out=destination_folder)

# Unzip the files
with zipfile.ZipFile(file1, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

with zipfile.ZipFile(file2, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)
    
with zipfile.ZipFile(file3, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

# Remove the zip files if needed
os.remove(file1)
os.remove(file2)
os.remove(file3)

Note: you may need to restart the kernel to use updated packages.


## Ingest 

In [5]:
# Find all jpgs and pngs in a folder and returns a list of their respective path 
def get_frame_paths(folder_path):
    frame_paths = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            frame_path = os.path.join(folder_path, filename)
            frame_paths.append(frame_path)
    frame_paths.sort()
    print(frame_paths)
    return frame_paths

In [6]:
# Checks if the inut file is a directory and if it is it will try and get the frames inside the folder
# it will check if it is and mp4 and if it is it will read it and create a list of frames
def get_input_frames(input_path):
    frames = []
    if os.path.isdir(input_path):
        frame_paths = get_frame_paths(input_path)
        print(f"{frame_paths}")
        frames = [cv2.imread(frame_path) for frame_path in frame_paths]
    else :
        print("here")
        if input_path.endswith('.mp4') or input_path.endswith('.mov') :
            cap = cv2.VideoCapture(input_path)
            frames = []
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                frames.append(frame)
            cap.release()
    return frames

## Load Algo Config from file

In [7]:
# Loads a yaml file with the parameters to run the code
def load_config(filename):
    with open(filename, 'r') as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
        config['input_path']= os.getcwd()+config['input_path'] # Fix to run in the location it called
        print(config)
    return config

## Flo file format Aux Functions

In [8]:
""" import struct

def write_flow_field(flow, filename):
    height, width, _ = flow.shape
    with open(filename, 'wb') as f:
        # Write the magic number: 'PIEH' in ASCII
        f.write(b'PIEH')

        # Write the width and height of the flow field
        f.write(np.array(width).astype(np.int32).tobytes())
        f.write(np.array(height).astype(np.int32).tobytes())

        # Interleave the u and v values and write the flow field data
        flow_data = np.concatenate((flow[..., 0], flow[..., 1]), axis=1)
        f.write(flow_data.astype(np.float32).tobytes())

def read_flow_field(filename):
    with open(filename, 'rb') as f:
        # Read the magic number
        magic = f.read(4).decode()
        if magic != 'PIEH':
            raise ValueError('Invalid flow file format.')

        # Read the width and height of the flow field
        width = np.frombuffer(f.read(4), dtype=np.int32)[0]
        height = np.frombuffer(f.read(4), dtype=np.int32)[0]

        # Read the flow field data
        flow_data = np.frombuffer(f.read(), dtype=np.float32)
        flow = flow_data.reshape((height, width, 2))

    return flow
 """

" import struct\n\ndef write_flow_field(flow, filename):\n    height, width, _ = flow.shape\n    with open(filename, 'wb') as f:\n        # Write the magic number: 'PIEH' in ASCII\n        f.write(b'PIEH')\n\n        # Write the width and height of the flow field\n        f.write(np.array(width).astype(np.int32).tobytes())\n        f.write(np.array(height).astype(np.int32).tobytes())\n\n        # Interleave the u and v values and write the flow field data\n        flow_data = np.concatenate((flow[..., 0], flow[..., 1]), axis=1)\n        f.write(flow_data.astype(np.float32).tobytes())\n\ndef read_flow_field(filename):\n    with open(filename, 'rb') as f:\n        # Read the magic number\n        magic = f.read(4).decode()\n        if magic != 'PIEH':\n            raise ValueError('Invalid flow file format.')\n\n        # Read the width and height of the flow field\n        width = np.frombuffer(f.read(4), dtype=np.int32)[0]\n        height = np.frombuffer(f.read(4), dtype=np.int32)[0]\n

In [9]:
import struct
import numpy as np


def read_flow_field(file_path):
    with open(file_path, "rb") as f:
        # Read .flo header
        magic = struct.unpack('f', f.read(4))[0]
        width = struct.unpack('i', f.read(4))[0]
        height = struct.unpack('i', f.read(4))[0]

        # Ensure .flo file format
        assert magic == 202021.25, "Invalid .flo file format."

        # Read optical flow data
        flow_data = np.fromfile(f, np.float32, width * height * 2)
        flow_data = flow_data.reshape((height, width, 2))

    return flow_data


def write_flow_field(file_path, flow_data):
    height, width = flow_data.shape[:2]
    with open(file_path, "wb") as f:
        # Write .flo header
        f.write(struct.pack('f', 202021.25))
        f.write(struct.pack('i', width))
        f.write(struct.pack('i', height))

        # Write optical flow data
        flow_data.flatten().astype(np.float32).tofile(f)

In [10]:
def calculate_angular_error(flow_gt, flow_pred):
    flow_gt_norm = np.sqrt(np.sum(flow_gt ** 2, axis=2))
    flow_pred_norm = np.sqrt(np.sum(flow_pred ** 2, axis=2))
    dot_product = np.sum(flow_gt * flow_pred, axis=2)
    cos_theta = np.clip(dot_product / (flow_gt_norm * flow_pred_norm), -1.0, 1.0)
    angular_error = np.arccos(cos_theta)
    return angular_error

def calculate_endpoint_error(flow_gt, flow_pred):
    endpoint_error = np.sqrt(np.sum((flow_gt - flow_pred) ** 2, axis=2))
    return endpoint_error


def calculate_error_statistics(error):
    mean_error = np.mean(angular_error)
    std_error = np.std(angular_error)
    return mean_error, std_error


In [14]:
# Example usage
original_file = "/workspace/VC/assignment2/dataset/other-gt-flow/Urban2/flow10.flo"
new_file = "new.flo"

# Read the original .flo file
original_flow = read_flow_field(original_file)

# Write the original flow to a new .flo file
write_flow_field(new_file, original_flow)

# Read the new .flo file
new_flow = read_flow_field(new_file)

#Sanity check for write and read function

# Compare the original and new flow files
identical = np.array_equal(original_flow, new_flow)
print("The original and new files are identical:", identical)

angular_error = calculate_angular_error(original_flow, new_flow)
angular_mean, angular_std = calculate_error_statistics(angular_error)

print('Mean Angular Error:', angular_mean)
print('Standard Deviation of Angular Error:', angular_std)

endpoint_error = calculate_endpoint_error(original_flow, new_flow)
endpoint_mean, endpoint_std = calculate_error_statistics(endpoint_error)

print('Mean Endpoint Error:', endpoint_mean)
print('Standard Deviation of Endpoint Error:', endpoint_std)
os.remove(new_file)


The original and new files are identical: True
Mean Angular Error: 9.9700934e-05
Standard Deviation of Angular Error: 0.00017069293
Mean Endpoint Error: 9.9700934e-05
Standard Deviation of Endpoint Error: 0.00017069293


# Lucas-Kanade

In [15]:
print(f"Loading File \'config_LK.yml\'")

config = load_config('config_LK.yml')

frames = get_input_frames(config['input_path'])

feature_params = dict(
    maxCorners=config['max_corners'],
    qualityLevel=config['quality_level'],
    minDistance=config['min_distance'],
    blockSize=config['block_size']
)
lk_params = dict(
    winSize=(config['window_size'], config['window_size']),
    maxLevel=config['max_level'],
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, config['max_iterations'], config['epsilon'])
)
color = np.random.randint(0, 255, (100, 3))
old_frame = frames[0]
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

mask = np.zeros_like(old_frame)
height, width, channels = old_frame.shape
mask_flow = np.zeros((height,width,2))
print("DIM: ", mask_flow.shape)

for frame_num in frames[1:]:
    
    current_frame = frame_num
    frame_gray = cv2.cvtColor(frame_num, cv2.COLOR_BGR2GRAY)
    
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    print("p1: ", np.asarray(p1).shape)
    print(np.asarray(p1).shape)
    print(p1)
    if p1 is not None:
        
        good_new = p1[st==1]
        good_old = p0[st==1]
    
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        
    frame_with_flow = cv2.add(current_frame, mask)
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    

print("mask: ",mask.shape)
cv2.imwrite('flow.png', frame_with_flow)
write_flow_field(config['output_path'],mask)


"""flow_data = read_flow_field('flow_field.flo')  # Read the .flo file

# Compute the magnitude and angle of the flow vectors
magnitude, angle = cv2.cartToPolar(flow_data[..., 0], flow_data[..., 1])

# Visualize the flow field
hue = angle * 180 / np.pi / 2
saturation = np.ones_like(magnitude)
value = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
flow_visualization = cv2.cvtColor(cv2.merge((hue, saturation, value)).astype(np.uint8), cv2.COLOR_HSV2BGR)
#teste = cv2.merge((hue, saturation, value)).astype(np.uint8)
# Display the flow visualization
cv2.imshow('Optical Flow', flow_visualization)
cv2.waitKey(0)
cv2.destroyAllWindows()"""

Loading File 'config_LK.yml'
{'input_path': '/workspace/VC/assignment2/dataset/Forest_15_3b_Videvo.mp4', 'output_path': 'flow_lk.flo', 'max_corners': 100, 'quality_level': 0.3, 'min_distance': 7, 'block_size': 7, 'window_size': 15, 'max_level': 2, 'max_iterations': 10, 'epsilon': 0.03, 'ransac_threshold': 3.0}
here
DIM:  (1080, 1920, 2)
p1:  (100, 1, 2)
(100, 1, 2)
[[[3.68156219e+02 7.10328430e+02]]

 [[1.16362561e+03 8.38359680e+02]]

 [[8.34298279e+02 7.28396851e+02]]

 [[1.18328528e+03 1.70670898e+02]]

 [[4.93973816e+02 7.24812195e+02]]

 [[9.71268677e+02 7.96998169e+02]]

 [[8.71272888e+02 7.16045959e+02]]

 [[7.07275848e+01 7.09497864e+02]]

 [[1.96354034e+02 1.57802536e+02]]

 [[3.51777832e+02 3.84345551e+02]]

 [[4.77251862e+02 7.58999390e+02]]

 [[8.18294556e+02 7.19069153e+02]]

 [[1.34077911e+02 6.79186035e+02]]

 [[2.61421204e+01 1.08224243e+03]]

 [[4.13739929e+02 4.73398407e+02]]

 [[1.89844946e+03 9.07724854e+02]]

 [[8.96300293e+02 7.74259216e+02]]

 [[3.45229340e+01 1.

"flow_data = read_flow_field('flow_field.flo')  # Read the .flo file\n\n# Compute the magnitude and angle of the flow vectors\nmagnitude, angle = cv2.cartToPolar(flow_data[..., 0], flow_data[..., 1])\n\n# Visualize the flow field\nhue = angle * 180 / np.pi / 2\nsaturation = np.ones_like(magnitude)\nvalue = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)\nflow_visualization = cv2.cvtColor(cv2.merge((hue, saturation, value)).astype(np.uint8), cv2.COLOR_HSV2BGR)\n#teste = cv2.merge((hue, saturation, value)).astype(np.uint8)\n# Display the flow visualization\ncv2.imshow('Optical Flow', flow_visualization)\ncv2.waitKey(0)\ncv2.destroyAllWindows()"

In [ ]:
image = cv2.imread('dataset/other-data/Urban2/frame07.png')

flow = read_flow_field('dataset/other-gt-flow/Urban2/flow10.flo')

warped = cv2.remap(image, flow, None, cv2.INTER_LINEAR)

cv2.imshow('warp', warped)
cv2.waitKey(0)
cv2.destroyAllWindows

# Horn-Shunchk

In [ ]:
def computeHS(path, name1, name2, alpha, delta):
    
    print("Computing using: ", name1, " and ", name2)
    beforeImg = cv2.imread(os.path.join(path, name1), cv2.IMREAD_GRAYSCALE)
    afterImg = cv2.imread(os.path.join(path, name2), cv2.IMREAD_GRAYSCALE)

    if beforeImg is None:
        raise NameError("Can't find image: \"" + name1 + '\"')
    elif afterImg is None:
        raise NameError("Can't find image: \"" + name2 + '\"')

    beforeImg = cv2.imread(os.path.join(path, name1), cv2.IMREAD_GRAYSCALE).astype(float)
    afterImg = cv2.imread(os.path.join(path, name2), cv2.IMREAD_GRAYSCALE).astype(float)

    #removing noise
    beforeImg  = cv2.GaussianBlur(beforeImg, (5, 5), 0)
    afterImg = cv2.GaussianBlur(afterImg, (5, 5), 0)

    # set up initial values
    u = np.zeros((beforeImg.shape[0], beforeImg.shape[1]))
    v = np.zeros((beforeImg.shape[0], beforeImg.shape[1]))
    fx, fy, ft = get_derivatives(beforeImg, afterImg)
    avg_kernel = np.array([[1 / 12, 1 / 6, 1 / 12],
                            [1 / 6, 0, 1 / 6],
                            [1 / 12, 1 / 6, 1 / 12]], float)
    iter_counter = 0
    while True:
        iter_counter += 1
        u_avg = filter2(u, avg_kernel)
        v_avg = filter2(v, avg_kernel)
        p = fx * u_avg + fy * v_avg + ft
        d = 4 * alpha**2 + fx**2 + fy**2
        prev = u

        u = u_avg - fx * (p / d)
        v = v_avg - fy * (p / d)

        diff = np.linalg.norm(u - prev, 2)
        #converges check (at most 300 iterations)
        if  diff < delta or iter_counter > 300:
            # print("iteration number: ", iter_counter)
            break

    draw_quiver(u, v, beforeImg)

    return [u, v]

def show_image(name, image):
    print("take this image!!!")
    if image is None:
        return

    cv2.imshow(name, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def get_magnitude(u, v):
    print("getting magnitude...")
    scale = 3
    sum = 0.0
    counter = 0.0

    for i in range(0, u.shape[0], 8):
        for j in range(0, u.shape[1],8):
            counter += 1
            dy = v[i,j] * scale
            dx = u[i,j] * scale
            magnitude = (dx**2 + dy**2)**0.5
            sum += magnitude

    mag_avg = sum / counter

    return mag_avg

def draw_quiver(u,v,beforeImg):
    print("drawing...")
    scale = 3
    ax = plt.figure().gca()
    ax.imshow(beforeImg, cmap = 'gray')

    magnitudeAvg = get_magnitude(u, v)

    for i in range(0, u.shape[0], 8):
        for j in range(0, u.shape[1],8):
            dy = v[i,j] * scale
            dx = u[i,j] * scale
            magnitude = (dx**2 + dy**2)**0.5
            #draw only significant changes
            if magnitude > magnitudeAvg:
                ax.arrow(j,i, dx, dy, color = 'red')

    plt.draw()
    plt.show()
    
def get_derivatives(img1, img2):
    print("derivating...")
    #derivative masks
    x_kernel = np.array([[-1, 1], [-1, 1]]) * 0.25
    y_kernel = np.array([[-1, -1], [1, 1]]) * 0.25
    t_kernel = np.ones((2, 2)) * 0.25

    fx = filter2(img1,x_kernel) + filter2(img2,x_kernel)
    fy = filter2(img1, y_kernel) + filter2(img2, y_kernel)
    ft = filter2(img1, -t_kernel) + filter2(img2, t_kernel)

    return [fx,fy, ft]

In [ ]:
print(f"Loading File \'config_HS.yml\'")

config = load_config('config_HS.yml')

input_path = config['input_path']

#name = get_frame_paths(input_path)
frames = os.listdir(input_path)
#print(frames)
for i in range(0,len(frames)-1):
    print("CYCLE: ", i)
    computeHS(input_path, frames[i], frames[i+1], alpha=15, delta=10**-1)

In [ ]:
import cv2
import numpy as np
import yaml
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output
from ipywidgets import Image
from io import BytesIO
import PIL.Image

def display_image(image):
    # Convert the OpenCV image to a PIL image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_image = PIL.Image.fromarray(image)
    
    # Display the image using the Jupyter inline magic
    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG")
    display(Image(data=buffered.getvalue()))


# Load the tuning parameters from the config file
with open('config_HS.yml', 'r') as f:
    config = yaml.safe_load(f)

# Load the input frames
input_path = os.getcwd()+config['input_path']
input_frames = get_input_frames(input_path)

# Initialize the flow field for the first frame with zeros
prev_frame = input_frames[0]
flow = np.zeros((prev_frame.shape[0], prev_frame.shape[1], 2), dtype=np.float32)

# Set the Horn-Schunck parameters
alpha = config['alpha']
num_iterations = config['num_iterations']
epsilon = config['epsilon']

# Define the Gaussian pyramid levels
num_levels = config['num_levels']
pyramid_scale = config['pyramid_scale']

# Create an output widget for displaying images
output_widget = Image()

# Compute the optical flow for each level of the pyramid
for level in range(num_levels):
    # Downsample the input frames and flow field
    curr_frame = cv2.resize(prev_frame, None, fx=pyramid_scale, fy=pyramid_scale)
    curr_flow = cv2.resize(flow, None, fx=pyramid_scale, fy=pyramid_scale)

    # Compute the optical flow using the multi-channel Horn-Schunck algorithm
    for i in range(num_iterations):
        # Split the flow field into x and y components
        fx, fy = np.split(curr_flow, 2, axis=2)

        # Compute the Laplacian of the flow field
        fxx, _ = np.gradient(fx)
        _, fyy = np.gradient(fy)
        fxy = np.gradient(fx, axis=0)[0] + np.gradient(fy, axis=1)[1]

        # Compute the temporal derivative of the flow field
        ft = next_frame_gray - prev_frame_gray + np.sum(curr_flow * np.dstack((fx, fy)), axis=2)

        # Compute the update to the flow field
        numerator = fxx * fy ** 2 - 2 * fx * fy * fxy + fyy * fx ** 2 - ft * fx * fy
        denominator = fx ** 2 + fy ** 2 + alpha
        update = numerator / (denominator[..., np.newaxis] + epsilon)

        # Update the flow field
        curr_flow += update

        # Visualize the current flow field
        if display:
            flow_image = flow_to_color(curr_flow)
            display_image(flow_image)

    # Convert the flow field to polar coordinates
    magnitude, angle = cv2.cartToPolar(curr_flow[..., 0], curr_flow[..., 1], angleInDegrees=True)

    #return curr_flow, magnitude, angle

# Visualization

Done Using [flowvid](https://github.com/diegoroyo/flowvid)

In [ ]:

import flowvid
import numpy as np
from IPython.display import HTML

import flowvid as fv

flo_data = fv.input.flo('/workspace/VC/assignment2/flow_lk.flo')

# You can normalize by frame OR the whole video
# Normalize each flo file independently
flo_frame = fv.normalize_frame(flo_data)
# Normalize all flo files at once, applying a clamp/gamma curve
flo_video = fv.normalize_video(flo_data, clamp_pct=0.8, gamma=1.5)

# Conversion from flow data to RGB
rgb_frames = fv.flow_to_rgb(flo_video)

out5 = fv.output.show_plot(title='Flow colors', framerate=10)
out5.show_all(rgb_frames, show_count=True)





In [ ]:
# IMport dataset with public ground truth
%pip install wget
import os
import subprocess
import wget
import zipfile

# Specify the URLs of the files to download
url1 = "https://vision.middlebury.edu/flow/data/comp/zip/other-gt-flow.zip"
url2 = "https://vision.middlebury.edu/flow/data/comp/zip/other-color-allframes.zip"
url3 = "https://vision.middlebury.edu/flow/data/comp/zip/eval-color-allframes.zip"
# Specify the destination folder to store the downloaded files
destination_folder = "dataset/"

# Download the files using wget
file1 = wget.download(url1, out=destination_folder)
file2 = wget.download(url2, out=destination_folder)
file3 = wget.download(url3, out=destination_folder)

# Unzip the files
with zipfile.ZipFile(file1, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

with zipfile.ZipFile(file2, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)
    
with zipfile.ZipFile(file3, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

# Remove the zip files if needed
os.remove(file1)
os.remove(file2)
os.remove(file3)

# Bench Marking


In [ ]:
import pandas as pd

def display_results():
    # Create an empty dataframe to store the results
    results_df = pd.DataFrame(columns=['DataSet', 'Average Angular Error '])
    
    # Iterate through the function calls and store the results
    for fc in fcalls:
        # Call the function
        result = fc()
        
        # Add the function name and result to the dataframe
        results_df = results_df.append({'Function': fc.__name__, 'Result': result}, ignore_index=True)
    
    # Display the results as a markdown table
    display(results_df.to_markdown(index=False))